<strong>1.</strong> Generați 2 imagini folosind Paint, similare cu Figura 1, care să conțină seturi de puncte în două sau mai multe culori – minimum 200 de puncte (așa cum este explicat în Descrierea detaliată a cerințelor), astfel încât în prima imagine setul de puncte să fie liniar separabil, iar în a doua imagine setul de puncte să fie non-liniar separabil. Salvați imaginile sub numele "data1.png" și "data2.png".

<strong>2.</strong> Creați două seturi de date pe baza celor 2 imagini din pasul 1, folosind codul dat mai jos. Analizați și comentați acest cod.
    

In [1]:
from PIL import Image
import numpy as np

def rgb_to_int(r, g, b):
    return (r << 16) + (g << 8) + b

def read_data(filename):
    x = []
    y = []
    back_color = rgb_to_int(255, 255, 255)  # Culoarea albă
    
    image = Image.open(filename)
    width, height = image.size
    pixels = image.load()
 
    for i in range(width):
        for j in range(height):
            pixel = pixels[i, j]  # Obține pixelul
            
            if len(pixel) == 3:  # Format RGB
                r, g, b = pixel
            elif len(pixel) == 4:  # Format RGBA
                r, g, b, _ = pixel  # Ignoră canalul alfa
            else:
                continue  # Sari peste pixeli care nu sunt RGB sau RGBA
            
            color = rgb_to_int(r, g, b)
            
            if color != back_color:  # Verifică dacă nu este alb
                x.append([i, j])  # Adaugă coordonatele pixelului
                y.append(color)  # Adaugă culoarea pixelului
    return x, y

x2, y2 = read_data('Non-linearly_separable.png')

# print("Data from Non-linearly Separable Image:")
# print("Coordinates:", x2)
# print("Colors:", y2)




<strong>3.</strong> Împarte primul set de date într-un set de antrenare și un set de testare (folosind 70% pentru antrenare și 30% pentru validare).


In [2]:
# impartirea setului de date a imaginii non-linearly separable
from sklearn.model_selection import train_test_split
X2_train, X2_test, y2_train, y2_test = train_test_split(x2, y2 , test_size= 0.3, random_state=42)

# print("Set de antrenare (X):", X2_train)
# print("Set de testare (X):", X2_test)
# print("Set de antrenare (y):", y2_train)
# print("Set de testare (y):", y2_test)


<strong>4.</strong> Alege un clasificator din categoria a) și antrenează-l pe setul de antrenare generat la pasul 3.



In [3]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()

model.fit(X2_train, y2_train)


GaussianNB()

<strong>5.</strong> Folosește clasificatorul antrenat la pasul 4 pentru a face predicții pe setul de testare generat la pasul 3.



In [4]:
# Predict Output pe setul de date a imaginii  non-linearly separable
model.fit(X2_train, y2_train)
predicted = model.predict([X2_train[6]])

print("Actual Value:", y2_train[6])
print("Predicted Value:", predicted[0])

Actual Value: 32768
Predicted Value: 32768


<strong>6.</strong> Calculează acuratețea clasificatorului pe setul de testare generat la pasul 3 și apoi pe setul de antrenare. Discută rezultatele.

In [5]:
from sklearn.metrics import accuracy_score

# acuratetea pe setul de testare a imaginii liniar separabile
model.fit(X2_train, y2_train)
predicted = model.predict(X2_test)
accuracy_test = accuracy_score(y2_test, predicted)

print(f"Acuratetea pe setul de testare: {accuracy_test}")

# acuratetea pe setul de antrenare a imaginii liniar separabile
predicted = model.predict(X2_train)
accuracy_training = accuracy_score(y2_train, predicted)

print(f"Acuratetea pe setul de antrenare: {accuracy_training}")


Acuratetea pe setul de testare: 0.5804055410560128
Acuratetea pe setul de antrenare: 0.5858864027538726


<strong>7.</strong> Calculează precizia și rechemarea (recall) clasificatorului pe setul de testare generat la pasul 3 și salvează rezultatele într-un fișier sau afișează-le. Definește (teoretic) precizia și rechemarea. Discută rezultatele.


In [6]:
from sklearn.metrics import precision_score, recall_score
#  recall si precizia pe setul de testare a imaginii non-liniar separabile
model.fit(X2_train, y2_train)
y_pred = model.predict(X2_test)

precision = precision_score(y2_test, y_pred, average='micro') # daca nu pun micro imi da eroare
recall = recall_score(y2_test, y_pred, average='micro')

print("Precision:", precision)
print("Recall:", recall)

Precision: 0.5804055410560128
Recall: 0.5804055410560128


<strong>8.</strong> Prezice culoarea pentru toți pixelii primei imagini și salvează culorile prezise într-o nouă imagine folosind codul de mai jos (Cod parțial dat. Trebuie completat). Fii capabil să explici codul de mai jos.




In [7]:
from PIL import Image

image = Image.open('Non-linearly_separable.png')
width, height = image.size

def generate_pixel_coordinates():
    points = []
    for i in range (width):
        for j in range(height):
            points.append([i,j])
            
    return points
        
def getRGBfromI(RGBint):#convert int color code to rgb color code
    blue =  RGBint & 255
    green = (RGBint >> 8) & 255
    red =   (RGBint >> 16) & 255
    return red, green, blue

def get_image_colors(image):
    colors = []
    for x in range(width):
        for y in range(height):
            rgb = image.getpixel((x, y))  # Obține culoarea RGB a pixelului
            colors.append((rgb[0], rgb[1], rgb[2]))  # Salvează RGB în lista de culori
    return colors

def save_data(pixels, colors, output_filename="NonLinearly.png"):
    im = Image.new("RGB", (width, height))
    pix = im.load()
    for i in range(len(pixels)):
        pix[pixels[i][0], pixels[i][1]] = colors[i]  # Asociază pixelilor noile culori
    im.save(output_filename, "PNG")

pixels = generate_pixel_coordinates()
colors = get_image_colors(image)
save_data(pixels, colors)


<strong>9.</strong> Folosește validarea încrucișată k-fold, cu valori diferite de k, pentru a evalua modelul (de exemplu, k=3, 5, 10). Calculează acuratețea în validarea încrucișată și media acurateței. Raportează rezultatele pentru toate rulările și compară-le cu acuratețea obținută la Pasul 6.



<strong>10.</strong> Repetă pașii 3-9 pentru al doilea și al treilea clasificator.




<strong>11.</strong> Din documentația scikit-learn, studiază clasificatorul al doilea ales, selectează doi hiperparametri reprezentativi și repetă de cel puțin 2 ori pașii 4-8 pentru diferite valori ale acestor hiperparametri. Raportează rezultatele pentru toate rulările și compară-le.



<strong>12.</strong> Folosește căutarea în grilă (grid search) cu validare încrucișată pentru optimizarea hiperparametrilor clasificatorilor (pentru selecția modelului). Raportează parametrii optimi furnizați de căutare. Prezice culoarea pentru toți pixelii primei imagini folosind modelul cu parametrii optimi.




<strong>13.</strong> Repetă toți pașii de mai sus pentru a doua imagine (folosind aceiași clasificatori ca pentru prima imagine). Compară rezultatele obținute cu același clasificator pentru cazurile liniar și non-liniar, respectiv.



<strong>14.</strong> Compară rezultatele din mai multe perspective:

<li>Compară rezultatele obținute pentru imagini liniar și non-liniar cu fiecare clasificator.</li> <li>Compară rezultatele obținute cu diferiți clasificatori pentru aceeași imagine.</li> <li>Încearcă să explici diferențele în performanțele diferitelor modele (clasificatori).</li>

<strong>15.</strong> Realizează un studiu de sensibilitate privind selecția datelor (fișierele data1 și data2) din imaginile originale (vezi primul punct din Descrierea Detaliată a Cerințelor), folosind unul dintre clasificatorii utilizați anterior. Explicație: Se presupune că pixelii din seturile de antrenare sunt derivați dintr-o imagine reală pe care o cunoașteți teoretic. Ajustați pixelii selectați, ținând cont de forma imaginii reale.

Generează două fișiere suplimentare, data12 și data22, și folosește unul dintre cei trei clasificatori pentru a prezice culoarea pentru toți pixelii din prima și a doua imagine. Compară rezultatele obținute (folosind data1 față de data12 și data2 față de data22). Oferă o explicație pentru rezultatele observate.






<strong>16.</strong> Realizează un studiu privind influența dimensiunii seturilor de antrenare și testare pentru unul dintre clasificatorii la alegerea ta.




<strong>17.</strong> Pentru evaluare, trebuie să completezi toți pașii descriși mai sus, să raportezi și să compari rezultatele, să oferi o prezentare succintă a clasificatorilor aleși direct în Jupyter Notebook și să fii pregătit să răspunzi la întrebări legate de prezentare.



